#### Prep Data

In [33]:
# set wd
import os
project_root = '/Users/kgedney/Documents/georgetown/anly503/project/'
os.chdir(project_root)

In [116]:
# prep
import pandas as pd
from pandas.api.types import CategoricalDtype

import bokeh
from bokeh.io import output_notebook
from bokeh.models.callbacks import CustomJS
from bokeh.models import ColumnDataSource, HoverTool

from bokeh.layouts import widgetbox, column
from bokeh.models.widgets import RadioButtonGroup, Slider

from bokeh.plotting import figure, output_file, show

output_notebook()

Loading BokehJS ...

In [50]:
# load data
gdp           = pd.read_csv('data/gdp.csv')
log_gdp       = pd.read_csv('data/log_gdp.csv')
gdp_growth    = pd.read_csv('data/gdp_growth.csv')
df_electric   = pd.read_csv('data/electric.csv')
df_internet   = pd.read_csv('data/internet.csv')
df_cellphones = pd.read_csv('data/cellphones.csv')

# add cat types back
cat_type = CategoricalDtype(categories=['High income', 'Upper middle income', 
                                        'Lower middle income', 'Low income'], 
                            ordered=True)
for df in [gdp, df_electric, df_internet, df_cellphones, log_gdp, gdp_growth]:
    df['income_level'] = df['income_level'].astype(cat_type)

In [53]:
df_electric.columns

Index(['country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', 'country_code', 'income_level', 'delta_2000',
       'gdp_growth_2017'],
      dtype='object')

In [90]:
# add gdp growth levels in 2017 to each df
gdp_growth = gdp_growth.rename(columns={'2017': 'gdp_growth_2017'})

df_electric   = pd.merge(df_electric, gdp_growth[['country', 'gdp_growth_2017']], on='country', how='left')
df_internet   = pd.merge(df_internet, gdp_growth[['country', 'gdp_growth_2017']], on='country', how='left')
df_cellphones = pd.merge(df_cellphones, gdp_growth[['country', 'gdp_growth_2017']], on='country', how='left')

#### Plot 

In [ ]:
# make database with three telecom indicators and all years

# database = {}
    
# indicators = ['electricty', 'internet', 'cellphones']
# for indicator in indicators:
#     database[indicator] = {}
#     for genre in set(df_full.core_genre):
#         tmp = make_dataset_genre(df_full, genre=genre, qtr_type=qtr_type)
#         database[qtr_type][genre] = safe_cds(tmp)
    
#     return database

In [62]:
# # add radio button (maybe these can be panes instead)
# radio_button_group = RadioButtonGroup(
#         labels=["Option 1", "Option 2", "Option 3"], active=0)

In [136]:
# add make_dataset function
def make_dataset(df, year_selected):
    df_sub = df[['country', 'country_code', 'income_level', 'gdp_growth_2017', str(year_selected)]]
    df_sub = df_sub.rename(columns={str(year_selected): 'year_selected'})
    return ColumnDataSource(df_sub)

In [137]:
# add year selector
year_slider = Slider(start=1990, end=2016, value=1990, step=1, title="Choose a Year")

# def function to update it
def update(year_slider):
    
   # grab selected year
    year_selected = year_slider.value
  
    # create new ColumnDataSource
    new_src = make_dataset(df_electric, year_selected=year_selected)

    new_src.change.emit()
    
# set callback
year_slider.callback = CustomJS.from_py_func(update)

TypeError: object of type 'NoneType' has no len()

In [142]:
# set figure
p = figure()

# initialize dataset
src = make_dataset(df_electric, year_selected=1990)

# set lines
p.circle(x = 'year_selected', 
         y = 'gdp_growth_2017',
         source=src)

# add hover
TOOLTIPS = [
        ('Country', '@country'),
        ('Income Level', '@income_level'),
        ('GDP Growth Rate (2017)', '@gdp_growth_2017')]

p.add_tools(HoverTool(tooltips=TOOLTIPS, toggleable=False))

layout = column(widgetbox(radio_button_group, slider, width=500), p) 

show(layout)